In [1]:
import pandas as pd
import rdkit as rk
from PIL import Image  # Import PIL for visualization
from rdkit import Chem
from IPython.display import display
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.rdMolDescriptors import CalcNumHeteroatoms
import random #library for the generation of DNA Tags
from openpyxl import Workbook
from rdkit.Chem import SDMolSupplier, SDWriter
from rdkit.Chem import rdChemReactions
from ClickReaction import CuAAC
from rdkit.Chem.Draw import IPythonConsole  # Ensures images render properly

In [7]:
# Define the reactant and product SMILES
def rxn_to_smarts(rxn_file):
    # Load reaction from RXN file
    with open(rxn_file_path, "r") as file:
        rxn_block = file.read()
    
    reaction = rdChemReactions.ReactionFromRxnBlock(rxn_block)
    
    if reaction is None:
        raise ValueError("Invalid RXN file or unable to parse.")
    
    # Convert to SMARTS notation
    smarts_notation = rdChemReactions.ReactionToSmarts(reaction)
    
    return smarts_notation

# Example Usage:
rxn_file_path = "ketcher.rxn"  # Replace with your actual RXN file path
smarts = rxn_to_smarts(rxn_file_path)
print("SMARTS Notation:", smarts)

SMARTS Notation: *-[#7].*-[#6:4](-*)-[#35]>>*-[#6:4](-*)-[#7]-*.[#35]


In [8]:
# Function to generate a random DNA sequence of length 5
def generate_random_sequences(num_sequences=10, length=5):
    sequences = [''.join(random.choices("ACGT", k=length)) for _ in range(num_sequences)]
    return sequences

# Generate and print 10 random sequences of length 5
random_sequences = generate_random_sequences(10, 5)
for seq in random_sequences:
    print(seq)

ATGCC
CAGAA
CTCTA
CCTGA
CATCC
TAGCG
GCCAA
TGTGT
ATAAT
CCTTA


In [8]:
# List of building blocks in SMILES format
BB = [
    'C#CC(N)c1ccccc1', 'NCC1CC#CCCC1', 'C#CCC(N)c1ccccc1',
    'O=C(O)c2cccc(Oc1ccccc1)c2', 'O=C(O)CC1CCC(C(F)(F)F)CC1', 'O=C(O)C2CCN(c1ccccc1)C2', 
    'N=[N+]=NCC1CCCC1', 'N=[N+]=NCc1ccccc1', 'N=[N+]=NCC1CCCCCC1'
]

# Convert SMILES to RDKit Mol objects
molecules = []
for i, smiles in enumerate(BB):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mol.SetProp("_Name", f"BB_{i}")  # Assign a name (important for SDF)
        AllChem.Compute2DCoords(mol)  # Compute 2D coordinates
        molecules.append(mol)
    else:
        print(f"❌ Warning: Could not parse SMILES {smiles}")

# Write valid molecules to an SDF file
sdf_filename = "building_blocks.sdf"
with Chem.SDWriter(sdf_filename) as writer:
    for mol in molecules:
        writer.write(mol)

print(f"✅ Successfully wrote {len(molecules)} molecules to {sdf_filename}")

✅ Successfully wrote 9 molecules to building_blocks.sdf


In [84]:
# Define the electrophile (with the leaving group Br)
electrophile_smiles = 'C#CC(Br)c1ccccc1'
electrophile = Chem.MolFromSmiles(electrophile_smiles)

# Define the nucleophile (amine group with an ether group)
nucleophile_smiles = 'Nc1ccoc1'
nucleophile = Chem.MolFromSmiles(nucleophile_smiles)

# Define the SN2 reaction SMARTS pattern
reaction_smarts = '[C:1]-Br.[N:2]>>[C:1]-[N:2]'

# Create the reaction object from SMARTS
reaction = AllChem.ReactionFromSmarts(reaction_smarts)

# Perform the reaction
reactants = [electrophile, nucleophile]
product_sets = reaction.RunReactants(reactants)

# Extract the products
products = []
for product_set in product_sets:
    for product in product_set:
        products.append(product)

In [10]:
# Define the reactants as SMILES
substrate = Chem.MolFromSmiles("C#CC(Br)c1ccccc1")  # 1-bromo-2-phenylprop-1-yne
nucleophile = Chem.MolFromSmiles("Nc1ccoc1")  # 2-aminofuran

# Define the reaction using SMARTS
reaction = AllChem.ReactionFromSmarts(
    "[C:1]([Br:2])=[C:3]#[C:4].[N:5]>>[C:1]([N:5])=[C:3]#[C:4].[Br-:2]"
)

# Run the reaction
products = reaction.RunReactants((substrate, nucleophile))

# Save the products to an SDF file
writer = Chem.SDWriter("products.sdf")
for prod_set in products:
    for prod in prod_set:
        writer.write(prod)
writer.close()

print("Products saved to products.sdf")

Products saved to products.sdf


In [6]:
# Definir los nucleófilos y electrófilos
nucleofilos = [
    Chem.MolFromSmiles("Nc1ccoc1"),  # Nucleófilo con nitrógeno
    Chem.MolFromSmiles("Oc1ccoc1"),  # Nucleófilo con oxígeno
    Chem.MolFromSmiles("Sc1cc[nH]c1"),  # Nucleófilo con azufre
    Chem.MolFromSmiles("Oc1cc[nH]c1")  # Otro nucleófilo con oxígeno
]

electrofilos = [
    Chem.MolFromSmiles("C#CC(Br)c1ccccc1"),  # Electrónfilo 1
    Chem.MolFromSmiles("C#CC(Br)C1CC1"),     # Electrónfilo 2
    Chem.MolFromSmiles("C#CC(Br)C1CCCC1")    # Electrónfilo 3
]

# Definir la reacción de sustitución nucleofílica
reaction = AllChem.ReactionFromSmarts("[C:1][Br].[N,O,S:2]>>[C:1][*:2]")

# Crear una lista para almacenar los productos
productos = []

# Iterar sobre los nucleófilos y electrófilos
for nucleofilo in nucleofilos:
    for electrofilo in electrofilos:
        # Ejecutar la reacción
        productos_reaccion = reaction.RunReactants((electrofilo, nucleofilo))
        for producto in productos_reaccion:
            productos.append(producto[0])  # Agregar el producto a la lista

# Guardar los productos en un archivo SDF
output_file = "productos.sdf"
writer = SDWriter(output_file)

for producto in productos:
    Chem.SanitizeMol(producto)  # Asegurarse de que la molécula sea válida
    writer.write(producto)

writer.close()
print(f"Se han generado {len(productos)} productos y se han guardado en el archivo {output_file}.")

Se han generado 12 productos y se han guardado en el archivo productos.sdf.


In [19]:
# Definir las moléculas en formato SMILES
smiles_azida = "N=[N+]=NCC1CCCC1"  # Azida
smiles_alquino = "C#CC(Br)c1ccccc1"  # Alquino

# Convertir los SMILES a objetos Mol de RDKit
mol_azida = Chem.MolFromSmiles(smiles_azida)
mol_alquino = Chem.MolFromSmiles(smiles_alquino)

# Enumerar los átomos en las moléculas
for atom in mol_azida.GetAtoms():
    atom.SetProp("atomNote", str(atom.GetIdx()))  # Agregar el índice del átomo como nota
for atom in mol_alquino.GetAtoms():
    atom.SetProp("atomNote", str(atom.GetIdx()))  # Agregar el índice del átomo como nota

# Dibujar las moléculas con los átomos enumerados
img_azida = Draw.MolToImage(mol_azida, legend="Azida: N=[N+]=NCC1CCCC1")
img_alquino = Draw.MolToImage(mol_alquino, legend="Alquino: C#CC(Br)c1ccccc1")

# Mostrar las imágenes
img_azida.show()
img_alquino.show()

# Guardar las imágenes en archivos
img_azida.save("azida_enumerada.png")
img_alquino.save("alquino_enumerado.png")

print("Imágenes generadas y guardadas como 'azida_enumerada.png' y 'alquino_enumerado.png'.")

Imágenes generadas y guardadas como 'azida_enumerada.png' y 'alquino_enumerado.png'.


In [52]:
supp = Chem.SDMolSupplier('building_blocks.sdf')
for mol in supp:
    if mol is not None: mol.GetNumAtoms()

In [53]:
mols = [x for x in supp]
len(mols)

10

In [54]:
patt1= Chem.MolFromSmarts('[$([NX3;H2;!$(NC=O)]),$([#16X2H]),$([OX2H])][cr5;$([cr5]:1:[nr5,or5,sr5]:[cr5]:[cr5]:[nr5,or5,sr5]:1),$([cr5]:1:[cr5]:[nr5,or5,sr5]:[cr5]:[cr5]:1)]')
het5 = [x for x in mols if x.HasSubstructMatch(patt1)]

In [55]:
patt2= Chem.MolFromSmarts('[Br][#6]C#[CH1]')
alkynes = [x for x in mols if x.HasSubstructMatch(patt2)]

In [56]:
patt3= Chem.MolFromSmarts('[N;H0;$(N-[#6]);D2]=[N;D2]=[N;D1]')
azide = [x for x in mols if x.HasSubstructMatch(patt3)]

In [57]:
rxn=AllChem.ReactionFromSmarts('[#6;a;r5:1][$([NX3;H2;!$(NC=O)]),$([#16X2H]),$([OX2H]):2].[#35,#17]-[#6:3][C:4]#[C:5]>>[#6;a;r5:1][$([NX3;H]),$([#16X2]),$([OX2]):2]-[#6:3][C:4]#[C:5]')

In [58]:
prods1 = AllChem.EnumerateLibraryFromReaction(rxn,[het5,alkynes])
smis = list(set([Chem.MolToSmiles(x[0],isomericSmiles=True) for x in prods1]))

In [59]:
rxn2= AllChem.ReactionFromSmarts('[#6:7][C:6]#[CH1:5].[#6:4]-[#7:3]=[N+:2]=[#7-:1]>>[#6:4]-[#7:3]-1[#6:5]=[#6:6](-[#6:7])-[#7:1]=[#7:2]-1')
prods2 = AllChem.EnumerateLibraryFromReaction(rxn2,[[ Chem.MolFromSmiles(x) for x in smis ],azide])
smis2 = list(set([Chem.MolToSmiles(x[0],isomericSmiles=True) for x in prods2]))
len(smis2)

0